In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F  
import torchvision
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader,Dataset
import numpy as np
import pandas as pd
from PIL import Image
import os
import shutil
import collections
import pickle
from tqdm.autonotebook import tqdm,trange

from torch import nn, optim

from torchvision import datasets, transforms, models
# from torchvision.transforms.functional import InterpolationMode
# import random
# import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


In [2]:
torch.__version__

'1.7.1+cu110'

In [3]:
device = 'cuda:0'

In [4]:
# class CNN(nn.Module):
#     def __init__(self):
#         super(CNN,self).__init__()
#         self.conv1 = nn.Conv2d(3, 6, 5)
#         self.pool = nn.MaxPool2d(2, 2)
#         self.conv2 = nn.Conv2d(6, 16, 5)
#         self.fc1 = nn.Linear(16 * 5 * 5, 120)
#         self.fc2 = nn.Linear(120, 84)
#         self.fc3 = nn.Linear(84,10)
#         self.dropout = nn.Dropout(p=0.5)  

#     def forward(self,x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv2(x)))
#         x = x.view(-1, 16 * 5 * 5)
#         x = F.relu(self.fc1(self.dropout(x)))
#         x = F.relu(self.fc2(self.dropout(x)))
#         x = self.fc3(self.dropout(x))
#         return F.log_softmax(x, dim=1)  
 
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(self.dropout(x)))
        x = F.relu(self.fc2(self.dropout(x)))
        x = F.relu(self.fc3(self.dropout(x)))
        return F.log_softmax(x, dim=1)


# class AlexNet(nn.Module):
#     """
#     Neural network model consisting of layers propsed by AlexNet paper.
#     """
#     def __init__(self, num_classes=2):
#         """
#         Define and allocate layers for this neural net.
#         Args:
#             num_classes (int): number of classes to predict with this model
#         """
#         super().__init__()
#         # input size should be : (b x 3 x 227 x 227)
#         # The image in the original paper states that width and height are 224 pixels, but
#         # the dimensions after first convolution layer do not lead to 55 x 55.
#         self.net = nn.Sequential(
#             nn.Conv2d(in_channels=3, out_channels=96, kernel_size=11, stride=4),  # (b x 96 x 55 x 55)
#             nn.ReLU(),
#             nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),  # section 3.3
#             nn.MaxPool2d(kernel_size=3, stride=2),  # (b x 96 x 27 x 27)
#             nn.Conv2d(96, 256, 5, padding=2),  # (b x 256 x 27 x 27)
#             nn.ReLU(),
#             nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),
#             nn.MaxPool2d(kernel_size=3, stride=2),  # (b x 256 x 13 x 13)
#             nn.Conv2d(256, 384, 3, padding=1),  # (b x 384 x 13 x 13)
#             nn.ReLU(),
#             nn.Conv2d(384, 384, 3, padding=1),  # (b x 384 x 13 x 13)
#             nn.ReLU(),
#             nn.Conv2d(384, 256, 3, padding=1),  # (b x 256 x 13 x 13)
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=3, stride=2),  # (b x 256 x 6 x 6)
#         )
#         # classifier is just a name for linear layers
#         self.classifier = nn.Sequential(
#             nn.Dropout(p=0.5, inplace=True),
#             nn.Linear(in_features=(256 * 6 * 6), out_features=500),
#             nn.ReLU(),
#             nn.Dropout(p=0.5, inplace=True),
#             nn.Linear(in_features=500, out_features=20),
#             nn.ReLU(),
#             nn.Linear(in_features=20, out_features=num_classes),
#         )

#     def forward(self, x):
#         """
#         Pass the input through the net.
#         Args:
#             x (Tensor): input tensor
#         Returns:
#             output (Tensor): output tensor
#         """
#         x = self.net(x)
#         x = x.view(-1, 256 * 6 * 6)  # reduce the dimensions for linear layer input
#         return self.classifier(x)

#定义Alexnet网路结构
class AlexNet(torch.nn.Module):
    def __init__(self, num_classes=10, init_weights=False):
        super(AlexNet, self).__init__()
        self.layer1 = torch.nn.Sequential(torch.nn.Conv2d(3, 96, kernel_size=5, stride=2, padding=2),
                                          # raw kernel_size=11, stride=4, padding=2. For use img size 224 * 224.
                                          torch.nn.BatchNorm2d(96),
                                          torch.nn.ReLU(inplace=True),
                                          torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0))
        self.layer2 = torch.nn.Sequential(torch.nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2),
                                          torch.nn.BatchNorm2d(256),
                                          torch.nn.ReLU(inplace=True),
                                          torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0))
        self.layer3 = torch.nn.Sequential(torch.nn.Conv2d(256, 384, kernel_size=3, padding=1),
                                          torch.nn.BatchNorm2d(384),
                                          torch.nn.ReLU(inplace=True))
        self.layer4 = torch.nn.Sequential(torch.nn.Conv2d(384, 384, kernel_size=3, padding=1),
                                          torch.nn.BatchNorm2d(384),
                                          torch.nn.ReLU(inplace=True))
        self.layer5 = torch.nn.Sequential(torch.nn.Conv2d(384, 384, kernel_size=3, padding=1),
                                          torch.nn.BatchNorm2d(384),
                                          torch.nn.ReLU(inplace=True),
                                          torch.nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc1 = torch.nn.Sequential(torch.nn.Dropout(p=0.5),
                                       torch.nn.Linear(384 * 2 * 2, 4096),
                                       torch.nn.ReLU(inplace=True))
        self.fc2 = torch.nn.Sequential(torch.nn.Dropout(p=0.5),
                                       torch.nn.Linear(4096, 4096),
                                       torch.nn.ReLU(inplace=True))
        self.fc3 = torch.nn.Sequential(torch.nn.Dropout(p=0.5),
                                       torch.nn.Linear(4096, num_classes))
        if init_weights:
            self._initialize_weights()
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out


In [5]:
from random import random, randint

# transform = transforms.Compose([transforms.Resize((224, 224), interpolation=InterpolationMode.BICUBIC),
#                                 transforms.ToTensor()
#                                 ])

transform = transforms.Compose([
                                transforms.ToTensor()
                                ])

train_dataset = torchvision.datasets.CIFAR10(
            root="/home/featurize/data",
            train=True,
            download=True,
            transform=transform
        )
        
test_dataset = torchvision.datasets.CIFAR10(
            root="/home/featurize/data",
            train=False,
            download=True,
            transform=transform
        )

class DatasetSplit(Dataset):
    """An abstract Dataset class wrapped around Pytorch Dataset class.
    """

    def __init__(self, dataset, idxs):
        self.dataset = dataset
        self.idxs = [int(i) for i in idxs]

    def __len__(self):
        return len(self.idxs)

    def __getitem__(self, item):
        image, label = self.dataset[self.idxs[item]]
        return torch.tensor(image), torch.tensor(label)

# print('train len: {} \t test len: {}'.format(len(train_dataset),len(test_dataset)))

samp_idxs = list(range(30000))
test_idxs = list(range(40000,50000))
train_dl = DataLoader(DatasetSplit(train_dataset,samp_idxs),batch_size=2048,num_workers=6)
test_dl = DataLoader(DatasetSplit(train_dataset,test_idxs),batch_size=2048,num_workers=6)

# train_dataset.data = train_dataset.data[:30000]
# train_dataset.targets = train_dataset.targets[:3000]

# train_dl = DataLoader(train_dataset,batch_size=1024,num_workers=5)
# test_dl = DataLoader(test_dataset,batch_size=1024,num_workers=5)

# model = AlexNet().to(device)
model = CNN().to(device)

print('train len: {} \t test len: {}'.format(len(train_dl),len(test_dl)))

# optimizer = torch.optim.SGD(model.parameters(),lr=0.1,momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(),lr=0.2)

criterion=nn.CrossEntropyLoss()

def inference(model,dataloader):
        model = model
        model.eval()
        loss,total,correct = 0.0,0.0,0.0        
        for batch_idx, (images, labels) in enumerate(dataloader):
            images, labels = images.to(device), labels.to(device)

            # Inference
            outputs = model(images)
            # print(outputs)
            # print(labels)
            batch_loss = criterion(outputs, labels)
            loss += float(batch_loss.item())

            # Prediction
            _, pred_labels = torch.max(outputs, 1)
            pred_labels = pred_labels.view(-1)
            correct += torch.sum(torch.eq(pred_labels, labels)).item()
            total += len(labels)

        accuracy = correct/total
        return accuracy, loss

epoch = 301
with trange(1,epoch) as t:
    for e in t:
    # for e in tqdm(range(epoch)):
        model.train()
        for batch_idx,(data,target) in enumerate(train_dl):
            data,target = data.cuda(),target.cuda()
            log_probs = model(data)
            loss = criterion(log_probs,target)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        
        acc,loss = inference(model,test_dl)
        t.set_postfix(epoch=e,acc=acc,loss=loss)

        if e in [100,150,200,250,300]:
            path = '/home/featurize/result/models/white_box_target_test_lanet'+str(e)+'.pth.tar'
            torch.save(model.state_dict(),path)
            print('已经保存')



Files already downloaded and verified
Files already downloaded and verified
train len: 15 	 test len: 5


  0%|          | 0/300 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [5]:
selected_conv_layer_names = ['layer1.0','layer2.0','layer3.0','layer4.0','layer5.0',
                            'fc1.1','fc2.1','fc3.1']

device = 'cuda:0'

model = AlexNet().to(device)

transform = transforms.Compose([
                                transforms.ToTensor()
                                ])

train_dataset = torchvision.datasets.CIFAR10(
            root="/home/featurize/data",
            train=True,
            download=True,
            transform=transform
        )

train_dl = DataLoader(train_dataset,batch_size=1,num_workers=6)


hidden_layer_features = {}
def get_hidden_layer_features(name):
    def fhook(model, input, output):
        hidden_layer_features[name] = output
        print('forward',name,output.shape)
    return fhook

grad_layer_features = {}
def get_grad_lalyer_features(name):
    def bhook(model,input,output):
        grad_layer_features[name] = output
        print('backwoard',name,input[2].shape,input[1].shape,output[0].shape)
    return bhook

optimizer = torch.optim.SGD(model.parameters(),lr=0.001,momentum=0.9)
# optimizer = torch.optim.Adam(model.parameters(),lr=0.02,momentum=0.5)

criterion=nn.CrossEntropyLoss()

for batch_idx, (images, labels) in enumerate(train_dl):

    optimizer.zero_grad()
    
    print(batch_idx)
    images, labels = images.to(device), labels.to(device)

    for name, layer in model.named_modules():
        if name in selected_conv_layer_names:
            if isinstance(layer, torch.nn.modules.conv.Conv2d):
                layer.register_forward_hook(get_hidden_layer_features(name))
                layer.register_backward_hook(get_grad_lalyer_features(name))
            if isinstance(layer,torch.nn.modules.linear.Linear):
                layer.register_forward_hook(get_hidden_layer_features(name))
                layer.register_backward_hook(get_grad_lalyer_features(name))
    
    output = model(images)

    loss = criterion(output,labels)
    loss.backward()

    # for name, layer in model.named_modules():
    #     if name in selected_conv_layer_names:
    #         # if isinstance(layer, torch.nn.modules.conv.Conv2d):
    #         #     grad_layer_features[name] = layer.weight.grad
    #         if isinstance(layer,torch.nn.modules.linear.Linear):
    #             # grad_layer_features[name] = layer.weight.grad
    #             # print('backward',name,grad_layer_features[name].shape)
    #             print(name,layer.weight.grad)

    # if batch_idx==2:
    # print(hidden_layer_features)
    # print(grad_layer_features)
    # break


Files already downloaded and verified
0
forward layer1.0 torch.Size([1, 96, 16, 16])
forward layer2.0 torch.Size([1, 256, 8, 8])
forward layer3.0 torch.Size([1, 384, 4, 4])
forward layer4.0 torch.Size([1, 384, 4, 4])
forward layer5.0 torch.Size([1, 384, 4, 4])
forward fc1.1 torch.Size([1, 4096])
forward fc2.1 torch.Size([1, 4096])
forward fc3.1 torch.Size([1, 10])
backwoard fc3.1 torch.Size([4096, 10]) torch.Size([1, 4096]) torch.Size([1, 10])
backwoard fc2.1 torch.Size([4096, 4096]) torch.Size([1, 4096]) torch.Size([1, 4096])
backwoard fc1.1 torch.Size([1536, 4096]) torch.Size([1, 1536]) torch.Size([1, 4096])


IndexError: tuple index out of range

In [ ]:
model_path = '/home/featurize/result/models/server/iid_P_FedAvg_250.pth.tar'
model = AlexNet().to(device)
weight = torch.load(model_path,map_location=device)
model.load_state_dict(weight)

print(weight['layer1.0.weight'])

op = torch.optim.SGD(model.parameters(),lr=1)
cr = nn.CrossEntropyLoss()

out = model(torch.rand(1,3,32,32).cuda())
loss = cr(out,torch.tensor(1).unsqueeze(dim=0).cuda())
loss.backward()
op.step()

print(weight['layer1.0.weight'])


In [ ]:
class CnnForCnnLayer(nn.Module):
    def __init__(self,input_shape):
        super().__init__()
        self.dim1 = input_shape[0]
        self.dim2 = input_shape[1]
        self.dim3 = input_shape[2]

        self.dropout = nn.Dropout(0)
        self.conv1 = nn.Conv2d(in_channels=self.dim1,out_channels=100,kernel_size=(self.dim2,self.dim3),stride=(1,1))
        self.fc1 = nn.Linear(100,128)
        self.fc2 = nn.Linear(128,64)
    def forward(self,x):
        x = F.leaky_relu(self.conv1(self.dropout(x)))
        print(x.shape)
        x = torch.flatten(x,start_dim=1,end_dim=-1)
        print(x.shape)
        x = F.leaky_relu(self.fc1(self.dropout(x)))
        x = F.leaky_relu(self.fc2(self.dropout(x)))
        return x

class FcnForModel(nn.Module):
    def __init__(self, dim_in=128, dim_out=64):
        super().__init__() 
        self.fc1 = nn.Linear(dim_in, 128)
        self.fc2 = nn.Linear(128, dim_out)
        self.dropout = nn.Dropout(0)

    def forward(self, x):
        x = F.leaky_relu(self.fc1(self.dropout(x)))
        x = F.leaky_relu(self.fc2(self.dropout(x)))
        return x

input_shape=(6,28,28)
x = torch.rand(2,1)
model = FcnForModel(dim_in=1)
y = model(x)
y

In [ ]:
import torch

a = torch.rand(64,5,5)
b = torch.rand(64,5,5)
c= []
c.append(a)
c.append(b)

d = torch.cat(c)

d.shape